In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
import time

RAW_DATA_PATH = "./raw_data.csv"
IRRELEVANT_COLUMNS = ['Unnamed: 0', 'Address']
CLEANED_DATA_FILE_PATH = "./clean_data.csv"

In [3]:
#Import data
df = pd.read_csv(RAW_DATA_PATH)

#Start cleaning process
df.drop(IRRELEVANT_COLUMNS, axis=1, inplace=True)
df = df[df['City'] != "Not in a city"]
df.loc[df['City'] == 'Dentsville (Dents)', 'City'] = 'Dentsville'
df.loc[df['City'] == 'Lexington-Fayette (corporate name for Lexington)', 'City'] = 'Lexington'
df.loc[df['City'] == 'Indianapolis (Remainder)', 'City'] = 'Indianapolis'
df.loc[df['City'] == 'Calumet City (PU RR name Calumet Park (sta.))', 'City'] = 'Calumet City'

In [5]:
unique_cities = df[['City', 'State']].drop_duplicates()

In [9]:
# Create a geolocator instance with a custom user_agent
geolocator = Nominatim(user_agent="city_geocoder")

# Dictionary to store results
city_coordinates = {}

for i, row in unique_cities.iterrows():
    # Build the query string with city, state, and country
    query = f"{row['City']}, {row['State']}, USA"
    location = geolocator.geocode(query)
    
    if location:
        city_coordinates[row['City']] = (location.latitude, location.longitude)
        print(f"{query}: {location.latitude}, {location.longitude}")
    else:
        not_found.append(row['City'])
        print(f"Location not found for {query}")
    
    time.sleep(1)

Phoenix, Arizona, USA: 33.4484367, -112.074141
Scottsdale, Arizona, USA: 33.4942189, -111.926018
Tucson, Arizona, USA: 32.2228765, -110.974847
Concord, California, USA: 37.9768525, -122.0335624
Bethel Island, California, USA: 38.029033999999996, -121.64000881834863
San Pablo, California, USA: 37.9621457, -122.3455263
Pittsburg, California, USA: 38.0181745, -121.8901232
Calexico, California, USA: 32.6668134, -115.4963754
Bakersfield, California, USA: 35.3738712, -119.019463
Burbank, California, USA: 34.1812089, -118.307201
Los Angeles, California, USA: 34.0536909, -118.242766
Long Beach, California, USA: 33.7690164, -118.191604
Hawthorne, California, USA: 33.9128272, -118.3426122
Costa Mesa, California, USA: 33.6633386, -117.903317
Rubidoux, California, USA: 33.99431195, -117.42376599221046
Arden-Arcade, California, USA: 38.6039613, -121.38300352999352
Victorville, California, USA: 34.5361067, -117.2911565
Chula Vista, California, USA: 32.6400541, -117.084195
San Diego, California, USA:

In [ ]:
city_coordinates

In [11]:
latitude = []
longitude = []

for city in df.City:
    (lat, long) = city_coordinates[city]
    longitude.append(long)
    latitude.append(lat)

In [19]:
df['Latitude'] = latitude

In [21]:
df['Longitude'] = longitude

In [35]:
# Convert "Date Local" to datetime format
df["Date"] = pd.to_datetime(df["Date"], errors='coerce', infer_datetime_format=True)

# Extract Year and Month, ensuring they are integers
df["Year"] = df["Date"].dt.year.astype("Int64")   # Nullable integer type
df["Month"] = df["Date"].dt.month.astype("Int64")  # Ensures no floating-point month
df["Day"] = df["Date"].dt.day.astype("Int64")  # Ensures no floating-point month

C:\Users\leona\AppData\Local\Temp\ipykernel_8536\105525534.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["Date"] = pd.to_datetime(df["Date"], errors='coerce', infer_datetime_format=True)


,Date,State,County,City,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO Mean,CO 1st Max Value,...,SO2 AQI,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,Latitude,Longitude,Year,Month,Day
0,2000-01-01,Arizona,Maricopa,Phoenix,0.019765,0.040,10,37,0.878947,2.2,...,13.0,19.041667,49.0,19,46,33.448437,-112.074141,2000,1,1
1,2000-01-02,Arizona,Maricopa,Phoenix,0.015882,0.032,10,30,1.066667,2.3,...,4.0,22.958333,36.0,19,34,33.448437,-112.074141,2000,1,2
2,2000-01-03,Arizona,Maricopa,Phoenix,0.009353,0.016,9,15,1.762500,2.5,...,16.0,38.125000,51.0,8,48,33.448437,-112.074141,2000,1,3
3,2000-01-04,Arizona,Maricopa,Phoenix,0.015882,0.033,9,31,1.829167,3.0,...,23.0,40.260870,74.0,8,72,33.448437,-112.074141,2000,1,4
4,2000-01-05,Arizona,Maricopa,Phoenix,0.007353,0.012,9,11,2.700000,3.7,...,21.0,48.450000,61.0,22,58,33.448437,-112.074141,2000,1,5
5,2000-01-06,Arizona,Maricopa,Phoenix,0.014000,0.025,10,23,2.308333,3.6,...,24.0,39.950000,73.0,8,71,33.448437,-112.074141,2000,1,6
6,2000-01-07,Arizona,Maricopa,Phoenix,0.013235,0.024,10,22,1.829167,3.5,...,30.0,29.625000,43.0,9,41,33.448437,-112.074141,2000,1,7
7,2000-01-08,Arizona,Maricopa,Phoenix,0.011176,0.020,10,19,2.787500,5.1,...,26.0,29.666667,41.0,0,39,33.448437,-112.074141,2000,1,8
8,2000-01-09,Arizona,Maricopa,Phoenix,0.012588,0.022,10,20,1.675000,2.8,...,19.0,25.083333,37.0,20,35,33.448437,-112.074141,2000,1,9
9,2000-01-10,Arizona,Maricopa,Phoenix,0.010000,0.015,9,14,2.179167,3.7,...,30.0,37.666667,70.0,20,68,33.448437,-112.074141,2000,1,10


In [39]:
df.head()

,State,County,City,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,...,SO2 AQI,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,Latitude,Longitude,Year,Month,Day
0,Arizona,Maricopa,Phoenix,0.019765,0.040,10,37,0.878947,2.2,23,...,13.0,19.041667,49.0,19,46,33.448437,-112.074141,2000,1,1
1,Arizona,Maricopa,Phoenix,0.015882,0.032,10,30,1.066667,2.3,0,...,4.0,22.958333,36.0,19,34,33.448437,-112.074141,2000,1,2
2,Arizona,Maricopa,Phoenix,0.009353,0.016,9,15,1.762500,2.5,8,...,16.0,38.125000,51.0,8,48,33.448437,-112.074141,2000,1,3
3,Arizona,Maricopa,Phoenix,0.015882,0.033,9,31,1.829167,3.0,23,...,23.0,40.260870,74.0,8,72,33.448437,-112.074141,2000,1,4
4,Arizona,Maricopa,Phoenix,0.007353,0.012,9,11,2.700000,3.7,2,...,21.0,48.450000,61.0,22,58,33.448437,-112.074141,2000,1,5


In [ ]:
df.to_csv(CLEANED_DATA_FILE_PATH)